# Fine-tuning a pre-trained BERT model for classification using native PyTorch
Competition: [Covid-19 tweet classification](https://zindi.africa/competitions/covid-19-tweet-classification)

## Installing libraries

In [1]:
# %pip install -r requirements.txt

## Importing Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

## Reading the data
Substitute the file paths with the paths for your own data.

In [3]:
data = pd.read_csv('./data/Train.csv')
test_data = pd.read_csv('./data/Test.csv')

In [4]:
data.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [5]:
data['target'].value_counts()

target
0    2746
1    2541
Name: count, dtype: int64

## Splitting the data
Train size = 70% of the total size
Test size = 30% of the total size

In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(data['text'], data['target'], train_size = 0.7, shuffle = True, random_state=42)

## Data Preparation

Initialize the tokenizer and pass the text data to get tokens that can be passed to the BERT model.

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [9]:
train_tokens = tokenizer(list(train_X), padding = True, truncation=True)
test_tokens = tokenizer(list(test_X), padding = True, truncation=True)

In [10]:
train_tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [11]:
print(train_tokens['input_ids'][0])
print(tokenizer.decode(train_tokens['input_ids'][0]))

[101, 3725, 9216, 8653, 2740, 124, 5187, 1559, 5465, 1545, 1367, 5966, 1580, 9493, 2740, 122, 5539, 1571, 26516, 4859, 1545, 4735, 1116, 21606, 5117, 1477, 3993, 1604, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] Update Total cases 3 017 766 12 879 Current cases 1 915 580 856 Deaths 207 722 468 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [12]:
print(train_tokens['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Create a custom Dataset class

In [13]:
class TokenData(Dataset):
    def __init__(self, train = False):
        if train:
            self.text_data = train_X
            self.tokens = train_tokens
            self.labels = list(train_Y)
        else:
            self.text_data = test_X
            self.tokens = test_tokens
            self.labels = list(test_Y)

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        sample = {}
        for k, v in self.tokens.items():
            sample[k] = torch.tensor(v[idx])
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

Setting batch size. Create Dataset class objects for training and testing data. Declare Dataloader objects for these Dataset objects.

In [14]:
batch_size = 40
train_dataset = TokenData(train = True)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = TokenData(train = False)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

Iterating through the train data loader

In [15]:
train_iter = iter(train_loader)
sample = next(train_iter)
print(sample.items())

dict_items([('input_ids', tensor([[  101,  7444,   153,  ...,     0,     0,     0],
        [  101, 20898, 14569,  ...,     0,     0,     0],
        [  101,   170,  2528,  ...,     0,     0,     0],
        ...,
        [  101, 21748,   157,  ...,     0,     0,     0],
        [  101,  3446,   188,  ...,     0,     0,     0],
        [  101,   153,  6447,  ...,     0,     0,     0]])), ('token_type_ids', tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])), ('attention_mask', tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])), ('labels', tensor([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 

## BERT Model, Optimizer Function, and Loss Function

We will declare the model, the optimizer function used to optimize the model, and the loss function that is to be minimized as part of the training phase.

In [16]:
# configuration = BertConfig(hidden_dropout_prob=0.3, num_hidden_layers = 12, attention_probs_dropout_prob = 0.4)
bert_model = BertForSequenceClassification.from_pretrained('bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
optimizer = AdamW(bert_model.parameters(), lr=1e-5)

In [18]:
loss_fn = torch.nn.CrossEntropyLoss()

## Training and testing blocks

In [19]:
num_epochs = 3
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [20]:
bert_model.to(device) # Transfer model to GPU if available

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
for epoch in range(num_epochs):
    print("Epoch: ",(epoch + 1))
    # TRAINING BLOCK STARTS
    bert_model.train()
    for i,batch in enumerate(train_loader):    
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Setting the gradients to zero
        optimizer.zero_grad()
        
        # Passing the data to the model
        outputs = bert_model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])
        
        # The logits will be used for measuring the loss
        pred = outputs.logits
        loss = loss_fn(pred, batch['labels'])

        # Calculating the gradient for the loss function
        loss.backward()
        
        # Optimizing the parameters of the bert model
        optimizer.step()

        # Calculating the running loss for logging purposes
        train_batch_loss = loss.item()
        train_last_loss = train_batch_loss / batch_size

        print('Training batch {} last loss: {}'.format(i + 1, train_last_loss))
    # Logging epoch-wise training loss
    print(f"\nTraining epoch {epoch + 1} loss: ",train_last_loss)
    # TRAINING BLOCK ENDS 

    # TESTING BLOCK STARTS
    bert_model.eval()
    correct = 0
    test_pred = []
    for i, batch in enumerate(test_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # We don't need gradients for testing
        with torch.no_grad():
            outputs = bert_model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])
        
        # Logits act as predictions
        logits = outputs.logits
        
        # Calculating total batch loss using the logits and labels
        loss = loss_fn(logits, batch['labels'])
        test_batch_loss = loss.item()
        
        # Calculating the mean batch loss
        test_last_loss = test_batch_loss / batch_size
        print('Testing batch {} loss: {}'.format(i + 1, test_last_loss))
        
        # Comparing the predicted target with the labels in the batch
        correct += (logits.argmax(1) == batch['labels']).sum().item()
        print("Testing accuracy: ",correct/((i + 1) * batch_size))
    
    print(f"\nTesting epoch {epoch + 1} last loss: ",test_last_loss)
    # TESTING BLOCK ENDS

Epoch:  1
Training batch 1 last loss: 0.017877323925495146
Training batch 2 last loss: 0.018201589584350586
Training batch 3 last loss: 0.017984268069267274
Training batch 4 last loss: 0.017446587979793548
Training batch 5 last loss: 0.017330190539360045
Training batch 6 last loss: 0.01730782687664032
Training batch 7 last loss: 0.016722582280635834
Training batch 8 last loss: 0.01714702248573303
Training batch 9 last loss: 0.016864123940467834
Training batch 10 last loss: 0.01759873330593109
Training batch 11 last loss: 0.016695693135261536
Training batch 12 last loss: 0.016329213976860046
Training batch 13 last loss: 0.01644885540008545
Training batch 14 last loss: 0.01563907265663147
Training batch 15 last loss: 0.017045900225639343
Training batch 16 last loss: 0.016291584074497222
Training batch 17 last loss: 0.017157545685768126
Training batch 18 last loss: 0.016683897376060484
Training batch 19 last loss: 0.016242915391922
Training batch 20 last loss: 0.015635231137275697
Trainin

In [22]:
print('Validation accuracy: ',correct/len(test_X))

Validation accuracy:  0.8947700063011972


Saving the model

In [24]:
torch.save(bert_model.state_dict(), "./model/model.pt")

## Testing the model (Optional)

In [25]:
test_data

,ID,text
0,test_2,Why is explained in the video take a look
1,test_3,Ed Davey fasting for Ramadan No contest
2,test_4,Is Doja Cat good or do you just miss Nicki Minaj
3,test_8,How Boris Johnson s cheery wounded in action p...
4,test_9,Man it s terrible Not even a reason to get on ...
...,...,...
1957,test_2932,Fageeru meehaa geyga Bandah PUBLIC fundS amp G...
1958,test_2934,DFFN Diffusion Pharmaceuticals Announces Pre I...
1959,test_2936,I want to wish the Muslim members of Congress ...
1960,test_2937,You mean you don t believe there is a conspira...


Tokenizing test data

In [26]:
test_data_tokens = tokenizer(list(test_data['text']), padding = True, truncation=True)
len(test_data_tokens['input_ids'])

1962

Preparing Dataset class for test data

In [27]:
class TestData(Dataset):
    def __init__(self):
            self.text_data = test_data['text']
            self.tokens = test_data_tokens
    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        sample = {}
        for k, v in self.tokens.items():
            sample[k] = torch.tensor(v[idx])
        return sample

Declaring an object for test dataset and test data loader

In [28]:
test_data_dataset = TestData()
test_data_loader = DataLoader(test_data_dataset, batch_size=1) 

Loading the saved (trained) model 

In [29]:
weights = torch.load("./model/model.pt")
bert_model.load_state_dict(weights)
bert_model.to(device)

/tmp/ipykernel_32398/368546927.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load("./model/model.pt")


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Preparing the submission file

In [30]:
bert_model.eval()
result = []
for i, batch in enumerate(test_data_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = bert_model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])
    
    logits = outputs.logits
    result.extend(list(torch.nn.functional.softmax(logits, dim = 1).type(torch.float)))
result = [i[1].item() for i in result]

In [31]:
result_df = pd.DataFrame()
result_df['ID'] = test_data['ID']
result_df['target'] = result

In [32]:
result_df.to_csv("./submission1.csv", index = False)